# Geolocalización con TF-IPF

En esta notebook haremos un intento de geolocalización con los textos de los usuarios.

Pero haremos algo distinto: usaremos Term Frequency - Inverse Province Frequency (TF-IPF)


[Geolocation prediction in social media data by finding location indicative words](http://www.aclweb.org/anthology/C12-1064)

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pandas as pd

df_train = pd.read_json("../data/geoloc/users_train.json")
df_test = pd.read_json("../data/geoloc/users_test.json")

Hagamos lo siguiente:

- Entrenemos con unigramas una regresión logística para 
- Luego probemos con los regionalismos

Primero, partamos en train, test

In [2]:
df_train.groupby("provincia").count()


,text
provincia,
buenosaires,337
catamarca,341
chaco,331
chubut,328
cordoba,317
corrientes,345
entrerios,338
formosa,286
jujuy,339


## Palabras precalculadas

Carguemos antes las palabras que sabemos que ocurren una cantidad razonable de veces

In [3]:
%%time
from contrastes.processing import build_dataframe_from_users
from contrastes.processing import preprocess_raw_df


#word_df = build_dataframe_from_users(row for index, row in df_train.iterrows())

word_df = pd.read_csv("train_word_df_filtered.csv", index_col=0)
word_df = preprocess_raw_df(word_df, filter_words=(10, 2))

CPU times: user 1.1 s, sys: 180 ms, total: 1.28 s
Wall time: 1.62 s


/home/jmperez/projects/contrastes/contrastes/processing.py:185: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnas_palabras = cant_palabras
/home/jmperez/projects/contrastes/contrastes/processing.py:186: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnas_personas = cant_personas


In [4]:
word_df.sort_values(["cant_provincias", "cant_palabra"], ascending=[True, False], inplace=True)

word_df.iloc[:10][["cant_palabra", "cant_provincias"]]

,cant_palabra,cant_provincias
tiemposur,883.0,1
logroño,711.0,1
nihuil,450.0,1
chivil,332.0,1
ipauss,315.0,1
vallerga,291.0,1
asprodema,290.0,1
cdelu,244.0,1
calahorra,216.0,1
canicross,202.0,1


Veamos qué performance tiene usando 1000, 2000, 3000, y así...

In [5]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
from contrastes.text import tokenize

liw_vectorizer = CountVectorizer(
    tokenizer=tokenize,
    vocabulary=word_df.index)

X_train = liw_vectorizer.fit_transform(df_train["text"])
print("Vectorizing")
X_test = liw_vectorizer.transform(df_test["text"])

Vectorizing
CPU times: user 9min 35s, sys: 836 ms, total: 9min 36s
Wall time: 9min 36s


Ya las tenemos vectorizadas en el orden esperado!

In [6]:
from sklearn.preprocessing import LabelEncoder

province_encoder = LabelEncoder()

province_encoder.fit(df_train["provincia"].values)

y_train = province_encoder.transform(df_train["provincia"].values)
y_test = province_encoder.transform(df_test["provincia"].values)

In [7]:
%%time
from contrastes.classifiers import fit_classifiers

num_words_to_fit = list(range(250, 5000, 250)) + list(range(5000, 20000, 500))

ret = fit_classifiers(X_train, y_train, X_test, y_test, 
                      province_encoder=province_encoder,
                      range_num_words=num_words_to_fit, num_jobs=8)

Entrenando con 250 palabras
Entrenando con 3750 palabras
Entrenando con 3250 palabras
Entrenando con 2250 palabras
Entrenando con 1250 palabras
Entrenando con 1750 palabras
Entrenando con 750 palabras
Entrenando con 2750 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


250   palabras ----> accuracy 23.20 mean distance 892.8596
Entrenando con 500 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


750   palabras ----> accuracy 33.52 mean distance 772.9668
Entrenando con 1000 palabras


...

2250  palabras ----> accuracy 51.40 mean distance 545.232
Entrenando con 2500 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


3750  palabras ----> accuracy 60.12 mean distance 414.0228
Entrenando con 4000 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


3250  palabras ----> accuracy 59.32 mean distance 429.2428
Entrenando con 3500 palabras
500   palabras ----> accuracy 30.12 mean distance 815.73
Entrenando con 4250 palabras
1000  palabras ----> accuracy 36.88 mean distance 729.8344
Entrenando con 4750 palabras
1500  palabras ----> accuracy 47.44 mean distance 592.1148
Entrenando con 5500 palabras
2000  palabras ----> accuracy 50.60 mean distance 555.0476
Entrenando con 6500 palabras
2500  palabras ----> accuracy 53.12 mean distance 523.9244
Entrenando con 7500 palabras
3000  palabras ----> accuracy 58.28 mean distance 436.4284
Entrenando con 8500 palabras
4000  palabras ----> accuracy 60.44 mean distance 408.2564
Entrenando con 9500 palabras
3500  palabras ----> accuracy 59.76 mean distance 422.914
Entrenando con 10500 palabras
4250  palabras ----> accuracy 62.04 mean distance 346.054
Entrenando con 4500 palabras
4750  palabras ----> accuracy 64.04 mean distance 330.8656
Entrenando con 5000 palabras
5500  palabras ----> accuracy 63.92

2500 palabras dan un accuracy de 71%. BASTANTE BIEN. Luego disminuye la performance

In [8]:
for r in ret:
    num_words = r["num_words"]
    acc = r["accuracy"]
    md = r["mean_distance"]
    print("{:<5} palabras ----> accuracy {:.2f} mean distance {}".format(
        num_words, acc*100, md
    ))

250   palabras ----> accuracy 23.20 mean distance 892.8596
500   palabras ----> accuracy 30.12 mean distance 815.73
750   palabras ----> accuracy 33.52 mean distance 772.9668
1000  palabras ----> accuracy 36.88 mean distance 729.8344
1250  palabras ----> accuracy 42.68 mean distance 652.8676
1500  palabras ----> accuracy 47.44 mean distance 592.1148
1750  palabras ----> accuracy 49.72 mean distance 566.3372
2000  palabras ----> accuracy 50.60 mean distance 555.0476
2250  palabras ----> accuracy 51.40 mean distance 545.232
2500  palabras ----> accuracy 53.12 mean distance 523.9244
2750  palabras ----> accuracy 56.44 mean distance 447.7888
3000  palabras ----> accuracy 58.28 mean distance 436.4284
3250  palabras ----> accuracy 59.32 mean distance 429.2428
3500  palabras ----> accuracy 59.76 mean distance 422.914
3750  palabras ----> accuracy 60.12 mean distance 414.0228
4000  palabras ----> accuracy 60.44 mean distance 408.2564
4250  palabras ----> accuracy 62.04 mean distance 346.054
45

In [10]:
import pickle

pickle.dump(ret, open("res_tf_ipf.pkl", "wb"))
